# Data and Analysis Plan: Performance Predictor for Boston Airbnb Listings

## Team 6
- Helene Barbier (barbier.h@northeastern.edu)
- John Ciolfi (ciolfi.j@northeastern.edu)
- Parth Gala (gala.pa@northeastern.edu)
- Ethan Lee (lee.et@northeastern.edu)

## Project Goal:
This work will use datasets from [Kaggle.com](https://www.kaggle.com/airbnb/boston) to predict how well a theoretical listing in Boston will perform based on the existing listings nearby.

# Data

## Overview
We will clean the datasets from [Boston Airbnb Open Data](https://www.kaggle.com/airbnb/boston).

From these datasets, we can obtain characteristics of existing Airbnb listings, including:
- Availability
- Location
- Property Type
- Bedrooms
- Bathrooms
- Day, Week, and Month prices
- Fees
- Reviews
- Host response time/rate

### Pipeline Overview
- `clean_listings`
    - includes only relevant listing features needed to predict performance of a listing
    - used for primary listing and host information
- `clean_availability`
    - converts listed dates to datetime.date objects, filters out redundant columns
    - used to draw conclusions about when listing is most available (e.g. holidays)
- `clean_reviews`
    - includes only relevant review features needed to predict performance of a listing
    - used to more accurately find review frequency and correlation to time period (e.g. holidays)

### Pipeline

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
def get_host_response_time(resp_desc):
    """Approximately converts a qualitative description of host response time to a number
    
    Args:
        resp_desc (str): qualitative description of how long a host takes to response
        
    Returns:
        approx_time (int): approximate number of hours it takes host to respond
    """
    if 'within a' in resp_desc:
        time_desc = resp_desc[resp_desc.rindex(' ')+1:]
        if time_desc == 'hour':
            return 1
        elif time_desc == 'hours':
            return 5
        else: # case for 'day'
            return 15
    else:
        return 48

In [3]:
def clean_listings(df_listings):
    """ Filters listings DataFrame to only include relevant columns

    Args:
        df_listings (DataFrame): Collection of airbnb listings and their features

    Returns:
        df_listings_clean (DataFrame): Selectively filtered collection of airbnb listings
    """
    # create map of original column names to prettier names
    col_map = \
    {
        'id': 'Listing ID',
        'price': 'Nightly Price',
        'bedrooms': 'Bedrooms',
        'bathrooms': 'Bathrooms',
        'availability_365': 'Availability per Year (%)',
        'number_of_reviews': 'Number of Reviews',
        'review_scores_rating': 'Rating (/100)',
        'review_scores_accuracy': 'review_scores_accuracy (/10)',
        'reviews_per_month': 'Reviews Per Month',
        'neighbourhood_cleansed': 'Neighborhood',
        'property_type': 'Property Type',
        'room_type': 'Room Type',
        'host_response_time': 'Approximate Host Response Time (hours)',
        'host_response_rate': 'Host Response Rate',
        'host_is_superhost': 'Host is Superhost',
        'host_listings_count': 'Host Listings Count',
        'host_identity_verified': 'Host Identity Verified',
    }

    # add relevant columns to new dataframe
    df_listings_clean = pd.DataFrame()
    for col_name, new_name in col_map.items():
        df_listings_clean[new_name] = df_listings[col_name].copy()

    # set listing id as index, drop any rows without data
    df_listings_clean.set_index('Listing ID', inplace=True)
    df_listings_clean.dropna(axis=0, how='any', inplace=True)
    
    # replace f -> False, t -> True
    df_listings_clean.replace(['f', 't'], [True, False], inplace=True)
    
    # convert number of days available per year to a percentage
    df_listings_clean[col_map['availability_365']] = \
        df_listings_clean[col_map['availability_365']].map(lambda days: f'{(days/365*100):.2f}%')
    
    # convert qualitative host response time (e.g. 'within a few hours') to an approximate number
    df_listings_clean[col_map['host_response_time']] = \
        df_listings_clean[col_map['host_response_time']].map(get_host_response_time)
    
    return df_listings_clean

In [4]:
df_listings = pd.read_csv('listings.csv.zip')
df_listings = clean_listings(df_listings)
df_listings.head()

,Nightly Price,Bedrooms,Bathrooms,Availability per Year (%),Number of Reviews,Rating (/100),review_scores_accuracy (/10),Reviews Per Month,Neighborhood,Property Type,Room Type,Approximate Host Response Time (hours),Host Response Rate,Host is Superhost,Host Listings Count,Host Identity Verified
Listing ID,,,,,,,,,,,,,,,,
3075044,$65.00,1.0,1.0,98.36%,36,94.0,10.0,1.30,Roslindale,Apartment,Private room,1,100%,True,1,False
6976,$65.00,1.0,1.0,87.40%,41,98.0,10.0,0.47,Roslindale,Apartment,Private room,5,100%,False,1,False
1436513,$75.00,1.0,1.0,26.85%,1,100.0,10.0,1.00,Roslindale,House,Private room,5,100%,True,1,True
7651065,$79.00,1.0,1.5,91.51%,29,99.0,10.0,2.25,Roslindale,House,Private room,1,100%,False,1,False
12386020,$75.00,1.0,1.0,15.89%,8,100.0,10.0,1.70,Roslindale,Condominium,Private room,5,100%,False,2,False


In [5]:
def clean_calendar(df_calendar):
    """ Filters calendar DataFrame to only include days when listing available and converts date column to date objects

    Args:
        df_calendar (DataFrame): collection of dates available for listings

    Returns:
        df_available_days (DataFrame): collection of dates available for listings 
            with date strings converted to date objects
    """
    # create copy of df_calendar
    df_available_days = df_calendar.copy()

    # replace f -> False, t -> True, and only include days where the listing is available
    df_available_days.replace(['f', 't'], [True, False], inplace=True)
    df_available_days = df_available_days[df_available_days['available']]

    # delete redundant columns (already in df_listings)
    del df_available_days['price']
    del df_available_days['available']

    # convert date in string form to datetime.date object
    df_available_days['date'] = df_available_days['date'].map(lambda d: datetime.strptime(d, '%Y-%m-%d').date())

    # rename cols and set index as Listing ID
    rename_map = \
    {
        'listing_id': 'Listing ID',
        'date': 'Date'
    }
    df_available_days.rename(columns=rename_map, inplace=True)
    df_available_days.set_index('Listing ID', inplace=True)

    return df_available_days

In [6]:
df_calendar = pd.read_csv('calendar.csv.zip')
df_available_days = clean_calendar(df_calendar)
df_available_days.head()

,Date
Listing ID,
12147973,2017-09-05
12147973,2017-09-04
12147973,2017-09-03
12147973,2017-09-02
12147973,2017-09-01


In [7]:
def clean_reviews(df_reviews):
    """Filters df_reviews by removing unnecessary columns and converts date column to date objects
    
    Args:
        df_reviews (DataFrame): collection of reviews for a listing
        
    Returns:
        df_reviews_clean (DataFrame): cleaned collection of reviews for a listing; only includes
            relevant columns and date strings converted to date objects
    """
    # initialize cleaned reviews dataframe
    df_reviews_clean = df_reviews.copy()

    # map date in string format to datetime.date objects
    df_reviews_clean['date'] = df_reviews_clean['date'].map(lambda d: datetime.strptime(d, '%Y-%m-%d').date())

    # delete reviewer id and name (doesn't tell us any useful information for our purposes)
    del df_reviews_clean['id']
    del df_reviews_clean['reviewer_name']

    # create renaming columns map
    rename_map = \
    {
        'listing_id': 'Listing ID',
        'reviewer_id': 'Reviewer ID',
        'date': 'Date',
        'comments': 'Comments'
    }

    # rename columns, set index to listing ID
    df_reviews_clean.rename(columns=rename_map, inplace=True)
    df_reviews_clean.set_index('Listing ID', inplace=True)
    
    return df_reviews_clean

In [8]:
df_reviews = pd.read_csv('reviews.csv.zip')
df_reviews = clean_reviews(df_reviews)
df_reviews.head()

,Date,Reviewer ID,Comments
Listing ID,,,
1178162,2013-05-21,4298113,My stay at islam's place was really cool! Good...
1178162,2013-05-29,6452964,Great location for both airport and city - gre...
1178162,2013-06-06,6449554,We really enjoyed our stay at Islams house. Fr...
1178162,2013-06-15,2215611,The room was nice and clean and so were the co...
1178162,2013-06-16,6848427,Great location. Just 5 mins walk from the Airp...


### Visualizations

### Analysis Plan